In [1]:
!git config --global credential.helper store
!pip install accelerate
!git config --global user.name "Neelectric"
!git config --global user.email "Neel.R@web.de"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 7.9 MB/s eta 0:00:0000:01


In [2]:
token = input(f"Enter token: ")

In [3]:
print("started script")
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
print("imported...")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf", 
                                        #   token = token,
                                          cache_dir = "../hf_cache/")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-13b-chat-hf", 
                                            #  token = token,
                                             device_map = "auto",
                                             torch_dtype = torch.float16,
                                             cache_dir = "../hf_cache/")

print("model loaded!")

started script
imported...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

model loaded!


In [4]:
from time import time
max_new_tokens = 400
prompt = "In the era of generative AI, "
medmcqa2 = """
Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
Let's think step by step. To this end, let's first reflect on the question and which concepts will be relevant in answering it. Then, let's brainstorm on how concepts in the answer options relate back to it. Finally, let's evaluate each option in relation to the question, choosing one.
"""
tokenized_prompt = tokenizer.encode(medmcqa2, return_tensors = "pt").to('cuda')
time_before = time()
raw_output = model.generate(tokenized_prompt,
                            # max_new_tokens = max_new_tokens,
                            temperature = 0.01)
time_after = time()
time_taken = time_after - time_before
untokenized_output = tokenizer.decode(raw_output[0], skip_special_tokens = True)
num_generated = len(raw_output[0]) - len(tokenized_prompt[0])
print(untokenized_output)
print(f"newly generated {num_generated}")
print(f"{num_generated / time_taken} t/s")


Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
Let's think step by step. To this end, let's first reflect on the question and which concepts will be relevant in answering it. Then, let's brainstorm on how concepts in the answer options relate back to it. Finally, let's evaluate each option in relation to the question, choosing one.

Answer.
The correct answer is (B) Glycogen breakdown.

Explanation.
Low insulin to glucagon ratio is not seen in glycogen breakdown. Insulin and glucagon are hormones that regulate carbohydrate metabolism. Insulin promotes glycogen synthesis and inhibits glycogen breakdown, while glucagon stimulates glycogen breakdown and inhibits glycogen synthesis. A low insulin to glucagon ratio indicates that glycogen breakdown is more active than glycogen synthesis.

Therefore, option (B) Glycogen breakdown is the correct answer.

Note that options (A) Glycogen synthesi

In [6]:
medmcqa = """
Question:
Axonal transport is:
(A) Antegrade
(B) Retrograde
(C) Antegrade and retrograde
(D) None
Answer: Let's reflect on each response individually, without choosing whether it is correct yet. Then, discuss the responses in relation to the question, hypothesizing on whether it might be the right answer. Finally choose one from these candidates after considering them carefully.
"""

In [4]:
!conda install faiss -y

Channels:
 - defaults
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/miniconda3

  added / updated specs:
    - faiss


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    archspec-0.2.3             |     pyhd3eb1b0_0          47 KB
    ca-certificates-2024.3.11  |       h06a4308_0         127 KB
    certifi-2024.2.2           |  py310h06a4308_0         159 KB
    conda-24.3.0               |  py310h06a4308_0         957 KB
    cudatoolkit-11.8.0         |       h6a678d5_0       630.7 MB
    faiss-1.7.4                |py310cuda112hae2f2aa_0_cuda         1.5 MB  conda-forge
    libblas-3.9.0              |1_h86c2bf4_netlib         199 KB  conda-forge
    libfaiss-1.7.4             |cuda112hb18a002_0_cuda        68.0 MB  conda-forge
    libfaiss-avx2-1.7.4        |cuda112h1234567_0_cuda        68.1 MB  conda-f

In [4]:
### runs a benchmark 
### Written by Neel Rajani

from src.biollama import BioLlama
from src.retrieval import load_db
from src.run_benchmark import run_benchmark

# Defining the model
model_type = "Llama-2" # eg. "Llama-2", "BioLlama"
model_size = "7b" # eg. "7b", "13b", "70b"
model_state = "untrained" # eg. "untrained", "pre-trained", "fine-tuned"

# If model is of type BioLlama, further define retrieval components and other properties
if model_type == "BioLlama":
    db_name = "RCT200k" # eg. "RCT200k", "PMA_y2k", "PMC_y2k"
    retriever_name = "MedCPT"
    neighbour_length = 32
    RETRO_layer_ids = [15]
else:
    db_name = None
    retriever_name = None
    neighbour_length = None
    RETRO_layer_ids = None

# Defining the benchmark to use
benchmark_name = "MedMCQA" # eg. "BioASQ5b", "PubMedQA", "MedQA-4", "MedQA-5", "MedMCQA"
benchmark_start_idx = 10
num_questions = 1000

print("running benchmark")
# Run the benchmark
run_benchmark(
        model_type = model_type,
        model_size = model_size,
        model_state = model_state,
        benchmark_name = benchmark_name,
        benchmark_start_idx = benchmark_start_idx,
        num_questions = num_questions,
        db_name = db_name,
        retriever_name = retriever_name,
        neighbour_length = neighbour_length,
        RETRO_layer_ids = RETRO_layer_ids
        )

# Judge the results

# Record the results

running benchmark


NameError: name 'neighbour_length' is not defined